<a href="https://colab.research.google.com/github/wooje-jung/class-project/blob/main/AI%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%98%EB%B0%8D_%EA%B3%BC%EC%A0%9C3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers

In [ ]:
# 구글 드라이브 Colab 환경에 마운트하기
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import re

# 엑셀파일 불러오기
dataset_train1 = pd.read_excel('/content/drive/MyDrive/katalk_data/카카오톡 대화내용.xlsx')

# 정규식을 이용하여 이름, 시간, 채팅 내용 추출하기
pattern = r'(\S+)\s\[([^\]]+)\]\s(.+)'
dataset_train1[['이름', '시간', '채팅']] = dataset_train1['채팅'].str.extract(pattern)

# 시간 정보 삭제하기
new_data = dataset_train1.drop(columns=['시간'])

# 결과 데이터프레임 출력하기
print(new_data[['이름', '채팅']])


       이름                        채팅
0     정성현                우제야 생일축하해!
1     정성현                우제야 생일축하해!
2     정현철                정보)우제 생일아님
3     정성현                  오늘 29일인줄
4     구도현               난 내년에 축하할게~
...   ...                       ...
6333  정성현                   먼가 크리미한
6334  정현철                      ㄷㄷㄷㄷ
6335  정우제                        님들
6336  정우제  이거 채팅방내용 추출하는걸로 과제발표해도됨?
6337  정현철                  추출임 유출임?

[6338 rows x 2 columns]


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch

# 훈련된 모델과 토크나이저를 불러오기
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 임베딩을 저장할 리스트
chat_embeddings = []

# 채팅 메시지를 반복하고 임베딩을 저장하기
for chat_text in new_data['채팅']:
    inputs = tokenizer(chat_text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    chat_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    chat_embeddings.append(chat_embedding)

# chat_embeddings 리스트를 데이터프레임으로 변환하기
embedding_df = pd.DataFrame(chat_embeddings)

# 만들어진 임베딩을 원래 데이터프레임에 새 열로 추가하기
new_data = pd.concat([new_data, embedding_df], axis=1)


print(new_data)


                            채팅  이름         0         1         2         3  \
0                   우제야 생일축하해!   2 -0.509945 -0.293520  0.324045 -0.433693   
1                   우제야 생일축하해!   2 -0.509945 -0.293520  0.324045 -0.433693   
2                   정보)우제 생일아님   4 -0.431421 -0.409895  0.401063 -0.601982   
3                     오늘 29일인줄   2 -0.400465 -0.485173  0.289293 -0.685321   
4                  난 내년에 축하할게~   0 -0.365748 -0.245194  0.375380 -0.424923   
...                        ...  ..       ...       ...       ...       ...   
6333                   먼가 크리미한   2 -0.142867 -0.400498 -0.037008 -0.515675   
6334                      ㄷㄷㄷㄷ   4 -0.402784  0.145996 -0.379183 -0.277948   
6335                        님들   3 -0.204651 -0.638040  0.056954 -0.162104   
6336  이거 채팅방내용 추출하는걸로 과제발표해도됨?   3 -0.306189 -0.209753  0.316369 -0.478991   
6337                  추출임 유출임?   4 -0.411519 -0.578170  0.509110 -0.478210   

             4         5         6         7  ...       758    

In [ ]:
from transformers import AutoTokenizer

# BERT 모델과 토크나이저 불러오기
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 질문 설정하기
question = "성현이 뭐해?"

# 질문을 토큰 ID로 변환하기
input_ids = tokenizer.encode(question, return_tensors="pt")

# 모델을 사용하여 임베딩 생성하기
question_embedding = model(input_ids).last_hidden_state.mean(dim=1).squeeze().detach().numpy()

# 모든 채팅 메시지와의 코사인 유사도 계산하기
similarities = []
for chat_embedding in chat_embeddings:
    similarity = 1 - cosine(question_embedding, chat_embedding)
    similarities.append(similarity)

# 가장 유사한 문장의 인덱스 찾기
most_similar_index = similarities.index(max(similarities))

# 가장 유사한 문장을 말한 사람 찾기
most_similar_name = new_data['이름'][most_similar_index]

# 가장 유사한 문장 출력하기
most_similar_sentence = new_data['채팅'][most_similar_index]
print(f"가장 유사한 문장: {most_similar_sentence}")
print(f"가장 유사한 문장을 말한 사람: {most_similar_name}")


가장 유사한 문장: 성현이 울산옴?
가장 유사한 문장을 말한 사람: 3
